In [2]:
#mount the drive
from google.colab import drive
drive.mount('/content/mydrive/')

Mounted at /content/mydrive/


In [ ]:
! git clone https://github.com/dalmiaujjwal/Data_Wrangling_Tips_Tricks.git

Cloning into 'Data_Wrangling_Tips_Tricks'...
remote: Enumerating objects: 103, done.
remote: Counting objects: 100% (103/103), done.
remote: Compressing objects: 100% (82/82), done.
remote: Total 103 (delta 34), reused 76 (delta 16), pack-reused 0
Receiving objects: 100% (103/103), 78.21 KiB | 3.13 MiB/s, done.
Resolving deltas: 100% (34/34), done.


### pd.read_csv(parse_dates = ) to deal with datetime

In [ ]:
path = '/content/mydrive/MyDrive/Datetime/Data_Wrangling_Tips_Tricks/Scenario_7'

In [ ]:
import pandas as pd